In [ ]:
from IPython.utils import capture
from IPython.display import clear_output
import os

#@markdown **WebUI Version 설치한 버전에 맞게 선택해주세요**
TagVersion = 'v1.8.0' #@param ["v1.3.2", "v1.4.1", "v1.5.2", "v1.6.0", "v1.7.0", "v1.8.0", "v1.9.0"]

#@markdown **체크포인트(checkpoint) 다운로드 URL**
Checkpoint_Url = '' #@param {type:"string"}
#@markdown **체크포인트 저장 파일이름**
Checkpoint_Filename = 'checkpoint.safetensors' #@param {type:"string"}

#@markdown **ControlNet 기본 모델 다운로드**
#@markdown > *openpose, lineart, softedge, depth, canny, segment, ip2p, inpaint, tile, scribble*  
ControlNet = True #@param {type:"boolean"}

#@markdown **ControlNet 추가 모델 다운로드**
#@markdown > *mlsd, normalbae, suffle*  
ControlNet_Extra = False #@param {type:"boolean"}

#@markdown **IP-Adapter 모델 다운로드**
CotnrolNet_IPAdapter = False #@param {type:"boolean"}

#@markdown **TemporalNet 모델 다운로드**
#@markdown > *temporalnet*  
ControlNet_TemporalNet = False #@param {type:"boolean"}

#@markdown **T2I Adapter 모델 다운로드**
#@markdown > *style, color, sketch, zoedepth, recolor*  
ControlNet_T2I = False #@param {type:"boolean"}

#@markdown **AnimateDiff 모델 다운로드**
AnimateDiff = False #@param {type:"boolean"}

#@markdown **AnimateDiff 추가 모션 모델 (HumansMotion,3DMotion)**
AnimateDiff_Extra = False #@param {type:"boolean"}

#@markdown **LCM-LoRa 사용**
LCMLora = False #@param {type:"boolean"}

#@markdown **Temporal Kit 사용**
TemporalKit = False #@param {type:"boolean"}

#@markdown **TemporalNet Util 사용**
TemporalNetUtil = False #@param {type:"boolean"}

#@markdown -----
#@markdown *터널링*

#@markdown **Ngrok**
Ngrok_Key = "" #@param {type:"string"}

Workspace = 'install'
ControlNet_Model_Path = "/content/controlnet_models"
AnimateDiff_Model_Path = "/content/motion_models"

from google.colab import drive
drive.mount('/content/drive')

!apt -y install -qq aria2
!pip install -q torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 torchtext==0.15.2 torchdata==0.6.1 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.20
!pip install httpx==0.24.1

!mkdir -p {ControlNet_Model_Path}
!mkdir -p {AnimateDiff_Model_Path}

%cd /content/drive/MyDrive/{Workspace}

print('download...')
with capture.capture_output() as cap:
  if Checkpoint_Url:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "{Checkpoint_Url}" -d ./models/Stable-diffusion -o {Checkpoint_Filename}

  if ControlNet:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_openpose_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_lineart_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_softedge_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1p_sd15_depth_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_canny_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1e_sd15_tile_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11f1e_sd15_tile_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_ip2p_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/hr16/ControlNet-HandRefiner-pruned/resolve/main/control_sd15_inpaint_depth_hand_fp16.safetensors -d {ControlNet_Model_Path} -o control_sd15_inpaint_depth_hand_fp16.safetensors

  if ControlNet_Extra:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_seg_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15s2_lineart_anime_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_inpaint_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_scribble_fp16.safetensors

    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_mlsd_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11p_sd15_normalbae_fp16.safetensors
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -d {ControlNet_Model_Path} -o control_v11e_sd15_shuffle_fp16.safetensors

  if CotnrolNet_IPAdapter:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter_sd15.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-plus_sd15.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/models/ip-adapter-plus-face_sd15.bin -d {ControlNet_Model_Path} -o ip-adapter-plus-face_sd15.pth

  if ControlNet_TemporalNet:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalNet/resolve/main/diff_control_sd15_temporalnet_fp16.safetensors -d {ControlNet_Model_Path} -o diff_control_sd15_temporalnet_fp16.safetensors

  if ControlNet_T2I:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_style_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -d {ControlNet_Model_Path} -o t2iadapter_color_sd14v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd15v2.pth -d {ControlNet_Model_Path} -o t2iadapter_sketch_sd15v2.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_zoedepth_sd15v1.pth -d .{ControlNet_Model_Path} -o t2iadapter_zoedepth_sd15v1.pth
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/ioclab_sd15_recolor.safetensors -d {ControlNet_Model_Path} -o ioclab_sd15_recolor.safetensors  

  if AnimateDiff:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v14.ckpt -d {AnimateDiff_Model_Path} -o mm_sd_v14.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15.ckpt -d {AnimateDiff_Model_Path} -o mm_sd_v15.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/mm_sd_v15_v2.ckpt -d {AnimateDiff_Model_Path} -o mm_sd_v15_v2.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/guoyww/animatediff/resolve/main/v3_sd15_mm.ckpt -d {AnimateDiff_Model_Path} -o v3_sd15_mm.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/CiaraRowles/TemporalDiff/resolve/main/temporaldiff-v1-animatediff.ckpt -d {AnimateDiff_Model_Path} -o temporaldiff-v1-animatediff.ckpt
    !ln -s ./extensions/animatediff/model/mm_sd_v14.ckpt {AnimateDiff_Model_Path}/mm_sd_v14.ckpt
    !ln -s ./extensions/animatediff/model/mm_sd_v15.ckpt {AnimateDiff_Model_Path}/mm_sd_v15.ckpt
    !ln -s ./extensions/animatediff/model/mm_sd_v15_v2.ckpt {AnimateDiff_Model_Path}/mm_sd_v15_v2.ckpt
    !ln -s ./extensions/animatediff/model/v3_sd15_mm.ckpt {AnimateDiff_Model_Path}/v3_sd15_mm.ckpt
    !ln -s ./extensions/animatediff/model/temporaldiff-v1-animatediff.ckpt {AnimateDiff_Model_Path}/temporaldiff-v1-animatediff.ckpt

  if AnimateDiff_Extra:
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/174464 -d {AnimateDiff_Model_Path} -o improvedHumansMotion_refinedHumanMovement.ckpt
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://civitai.com/api/download/models/178017 -d {AnimateDiff_Model_Path} -o improved3DMotion_improved3DV1.ckpt
    !ln -s {AnimateDiff_Model_Path}/improvedHumansMotion_refinedHumanMovement.ckpt ./extensions/animatediff/model/improvedHumansMotion_refinedHumanMovement.ckpt
    !ln -s {AnimateDiff_Model_Path}/improved3DMotion_improved3DV1.ckpt ./extensions/animatediff/model/improved3DMotion_improved3DV1.ckpt

def clone_pull(git_url, ext_name):
  if os.path.isdir('./extensions/'+ext_name):
    %cd ./extensions/{ext_name}
    !git pull
    %cd ../..
  else:
    !git clone {git_url} ./extensions/{ext_name}

if LCMLora:
  if TagVersion in ["v1.3.2", "v1.4.1", "v1.5.2", "v1.6.0", "v1.7.0"]:
    clone_pull('https://github.com/neuralninja22/lcm-sampler', 'lcm-sampler')
  !wget https://huggingface.co/latent-consistency/lcm-lora-sdv1-5/resolve/main/pytorch_lora_weights.safetensors -O ./models/Lora/lcm-lora-sdv1-5.safetensors

if TemporalKit:
  clone_pull('https://github.com/CiaraStrawberry/TemporalKit', 'TemporalKit')
  !pip install scenedetect

if TemporalNetUtil:
  clone_pull('https://github.com/ninjaneural/temporalnet_util', 'temporalnet_util')
  !pip install ffmpeg-python

%cd /content/drive/MyDrive/install

if Ngrok_Key:
  !pip install pyngrok
  from pyngrok import conf, ngrok
  conf.get_default().auth_token = Ngrok_Key
  conf.get_default().monitor_thread = False
  ssh_tunnels = ngrok.get_tunnels(conf.get_default())
  if len(ssh_tunnels) == 0:
      ssh_tunnel = ngrok.connect(7860)
      print('address：'+ssh_tunnel.public_url)
  else:
      print('address：'+ssh_tunnels[0].public_url)

  !python launch.py --xformers --no-half-vae --enable-insecure-extension-access --theme dark --controlnet-dir {ControlNet_Model_Path}
else:
  !python launch.py --share --xformers --no-half-vae --enable-insecure-extension-access --theme dark --controlnet-dir {ControlNet_Model_Path}

